<a href="https://colab.research.google.com/github/siposroland/cvs_hw_3d/blob/master/HF_merge_all_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Systems Homework**




# **Readme**

 Please run all 3 sections:

*   Download elements and model
*   Run the sources
*   Run Evalutation with 3 task functions






# Your Work

In [0]:
# Homework dataset
!wget http://deeplearning.iit.bme.hu/CVS/HW.zip
!unzip -qq HW.zip
!rm HW.zip

# Traffic Sign Classification set
!wget http://deeplearning.iit.bme.hu/CVS/trafficSignsHW.zip
!unzip -qq trafficSignsHW.zip
!rm trafficSignsHW.zip

# Get model
!wget https://github.com/siposroland/cvs_hw_3d/raw/master/model.pth

In [0]:
# Task 1: (traditional vision)
# BASIC: Create an algorithm to accurately detect and classify the 3 objects of interest (Cactus, Vehicle, Traffic Sign). You don't have to determine the subclass at this point.
# OUTPUT: center, width, high and type of objects / pictures

# HARDCORE: Determine the subclasses of Cacti and Vehicles
# OUTPUT: center, width, high and type of objects + subclass of cacti and vehicles / pictures

import pickle
import cv2
import numpy as np
import argparse
import time
import os
import math
#from evaluate import evaluate

font = cv2.FONT_HERSHEY_COMPLEX

################################## different edge and contour detection
def getBlurredEdges(img):
  imgEq = cv2.equalizeHist(img)
  imgBlur = cv2.GaussianBlur(imgEq, (5,5),1)
  #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])    #élesítés
  #sharpenedImg = cv2.filter2D(imgBlur, -1, kernel)

  imgCanny = cv2.Canny(imgBlur, 50, 100)
  return imgCanny

def getSimpleEdges(img):
  imgCanny = cv2.Canny(img, 50, 100)
  return imgCanny

def getEqEdges(img):
  imgEq = cv2.equalizeHist(img)
  imgCanny = cv2.Canny(imgEq, 50, 100)
  return imgCanny

def getSharpenedEdges(img):
  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])    #élesítés
  imgBlur = cv2.GaussianBlur(img, (5,5),1)
  sharpenedImg = cv2.filter2D(imgBlur, -1, kernel)
  imgCanny = cv2.Canny(sharpenedImg, 50, 100)
  return imgCanny

def getOnlySharpenedEdges(img):
  imgEq = cv2.equalizeHist(img)
  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])    #élesítés
  sharpenedImg = cv2.filter2D(imgEq, -1, kernel)
  imgCanny = cv2.Canny(sharpenedImg, 100, 200)
  return imgCanny

def getOnlyBlurredEdges(img):
  imgBlur = cv2.GaussianBlur(img, (5,5),1)
  imgCanny = cv2.Canny(imgBlur, 50, 100)
  return imgCanny

def getContours(img):
  imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blurredEdges = getBlurredEdges(imgGray)
  contours=[]
  blurredContours, _ = cv2.findContours(blurredEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  contours.append(blurredContours)
  simpleEdges = getSimpleEdges(imgGray)
  simpleContours, _ = cv2.findContours(simpleEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  contours.append(simpleContours)
  eqEdges = getEqEdges(imgGray)
  eqContours, _ = cv2.findContours(eqEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  sharpenedEdges = getSharpenedEdges(imgGray)
  sharpenedContours, _ = cv2.findContours(sharpenedEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  contours.append(sharpenedContours)
  onlyBlurredEdges = getOnlyBlurredEdges(imgGray)
  onlyBlurredContours, _ = cv2.findContours(onlyBlurredEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  contours.append(onlyBlurredContours)
  #onlySharpenedEdges = getOnlySharpenedEdges(imgGray)
  #onlySharpenedContours, _ = cv2.findContours(onlySharpenedEdges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
  #contours.append(onlySharpenedContours)
  print("contour test:")
  contoursAndRectangles = []
  for contourGroup in contours:        #keresés az összes kontúr típusban
    for i in range(len(contourGroup)):
      cnt = contourGroup[i]
      contourLength = cv2.arcLength(cnt, True)
      contourArea = cv2.contourArea(cnt, False)
      approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
      #x = approx.ravel()[0]
      #y = approx.ravel()[1]
      if(contourLength > 60 and len(approx)==3):
        print("tri")
        #cv2.drawContours(full_img, [approx], 0, (0, 255, 0), 2)
        #cv2.putText(full_img, "Triangle", (x, y), font, 1, (0))
      elif(contourLength > 60 and len(approx)==4):
        #x, y, w, h = cv2.boundingRect(i)
        #contoursAndRectangles.append([x,y,w,h],[cnt])
        print("rect")
        #cv2.drawContours(full_img, [approx], 0, (0, 255, 0), 2)
        #cv2.putText(full_img, "Rectangle", (x, y), font, 1, (0))
      elif(contourLength > 60 and  contourArea> (contourLength**2 / (4*3.14) *0.5) and len(approx)==5 ):
        x, y, w, h = cv2.boundingRect(cnt)
        contoursAndRectangles.append([[x,y,w,h],[cnt],[approx], "Pentagon"])

      elif(contourLength > 60 and  contourArea> (contourLength**2 / (4*3.14) *0.5) and 6<=len(approx)<15):
        x, y, w, h = cv2.boundingRect(cnt)
        contoursAndRectangles.append([[x,y,w,h],[cnt],[approx], "Ellipse"])
      elif(contourLength > 60 and  contourArea> (contourLength**2 / (4*3.14) *0.5)):
        x, y, w, h = cv2.boundingRect(cnt)
        contoursAndRectangles.append([[x,y,w,h],[cnt],[approx], "Circle"])

  drawnContours = []


  for i in range(len(contoursAndRectangles)):
    checkIfLargerRect(contoursAndRectangles, i)


  for i in range(len(contoursAndRectangles)):
    if contoursAndRectangles[i][0] == [0, 0, 0, 0]: continue
    drawnContours.append(contoursAndRectangles[i])
    cv2.rectangle(img, (contoursAndRectangles[i][0][0], contoursAndRectangles[i][0][1]), 
    (contoursAndRectangles[i][0][0]+contoursAndRectangles[i][0][2], contoursAndRectangles[i][0][1]+contoursAndRectangles[i][0][3]), (0,255,0),1)
    #cv2.drawContours(img, contoursAndRectangles[i][2], 0, (0, 255, 0), 2)
    cv2.putText(img, contoursAndRectangles[i][3], (x, y), font, 1, (0))


  return onlyBlurredEdges, contoursAndRectangles


#a kisebb intersection-nel tér vissza
def getIntersectionPercent(rect1, rect2):
  if((rect1[2] * rect1[3]) == 0 or (rect2[2] * rect2[3]) == 0):
    return 0
  [x,y,w,h] = getIntersection(rect1, rect2)
  percent1 = ((w*h) / (rect1[2] * rect1[3])) *100
  percent2 = ((w*h) /(rect2[2] * rect2[3])) *100
  if(percent1>percent2): 
    return percent2
  else:
    return percent1


def getIntersection(rect1, rect2):
  x = max(rect1[0], rect2[0])
  y = max(rect1[1], rect2[1])
  w = min(rect1[0]+rect1[2], rect2[0]+rect2[2])-x
  h = min(rect1[1]+rect1[3], rect2[1]+rect2[3])-y
  if w<0 or h<0: return (0,0,0,0)
  return(x,y,w,h)


#ellenőrzi, hogy van-e olyan négyzet, ami tartalmazza ezt a négyzetet
#cél: a legnagyobb wrapper négyzet megtalálása
def checkIfLargerRect(contourAndRectangleList, pos):
  rectParams = contourAndRectangleList[pos][0]
  retval = False
  for i in range(len(contourAndRectangleList)):
    if(i == pos) or (contourAndRectangleList[i][0] == [0, 0, 0, 0]): continue
    if(contourAndRectangleList[i][0][0]==rectParams[0] and contourAndRectangleList[i][0][1]==rectParams[1] and 
      (contourAndRectangleList[i][0][0]+contourAndRectangleList[i][0][2]) ==(rectParams[0]+rectParams[2]) and
      (contourAndRectangleList[i][0][1]+contourAndRectangleList[i][0][3]) ==(rectParams[1]+rectParams[3])):
      contourAndRectangleList[i][0] = [0,0,0,0]
      retval = False
    elif(contourAndRectangleList[i][0][0]<=rectParams[0] and contourAndRectangleList[i][0][1]<=rectParams[1] and 
      (contourAndRectangleList[i][0][0]+contourAndRectangleList[i][0][2]) >=(rectParams[0]+rectParams[2]) and
      (contourAndRectangleList[i][0][1]+contourAndRectangleList[i][0][3]) >=(rectParams[1]+rectParams[3])):
      contourAndRectangleList[pos][0] = [0,0,0,0] 
      retval = True

    elif(getIntersectionPercent(rectParams, [contourAndRectangleList[i][0][0], 
    contourAndRectangleList[i][0][1], contourAndRectangleList[i][0][2], contourAndRectangleList[i][0][3]]) > 80):  #overlapping négyzet eltávolítása, ha esetleg van
      contourAndRectangleList[i][0] = [0,0,0,0] #másik négyzet eltávolítása
      retval = False
  return retval



#transform color 
def colorTransform(img):
  imgValMean = cv2.mean(img)

  print("img color val means: ")
  print(imgValMean)

  millis = int(round(time.time() * 1000))
  print(millis)
  
  image_data = np.asarray(img)
  selectorCheck = np.zeros((480, 640))     #mutatja, hogy mi alapján végzek szín szűrést
  greenInHighBlue = 0  #sok kéket tartalmazó színben szereplő zöld
  redInHighBlue = 0    #sok kéket tartalmazó színben szereplő piros
  blueInHighBlue = 0
  highBluePixels = 0   #sok kéket tartalmazó pixelek száma
  print("image data test:")
  print(len(image_data[0]))
  for i in range(len(image_data)):
    for j in range(len(image_data[0])):
      pixel = image_data[i][j] 
      if(pixel[0]>imgValMean[0]*1.3 and pixel[2]>imgValMean[0]*0.5):
        blueInHighBlue += pixel[0]
        greenInHighBlue += pixel[1]
        redInHighBlue += pixel[2]
        highBluePixels += 1
        selectorCheck[i][j] = 255
  
  greenInHighBlue = greenInHighBlue/highBluePixels
  redInHighBlue = redInHighBlue/highBluePixels
  blueInHighBlue = blueInHighBlue/highBluePixels

  print("blue in high blue:")
  print(blueInHighBlue)
  print("green in high blue:")
  print(greenInHighBlue)
  print("red in high blue:")
  print(redInHighBlue)


  multGreen = 200/greenInHighBlue
  multRed = 150/redInHighBlue
  multBlue = 150/blueInHighBlue


  ##itt beállítom a képet, ezt nem feltétlenül fogom megtenni, az éles alaklmazásban csak a színkeresési feltételeket transzformálom, most viszont jól jön
  for i in range(0,img.shape[0]):       #ez már nem annyira nagyonlassú
    for j in range(0,img.shape[1]):
      pixelBlue = img.item(i, j, 0)
      pixelGreen = img.item(i, j, 1)
      pixelRed = img.item(i, j, 2)
      img.itemset((i, j, 1), min(pixelGreen*multGreen, 255))
      img.itemset((i, j, 2), min(pixelRed*multRed, 255))
      img.itemset((i, j, 0), min(pixelBlue*multBlue, 255))


  millis = int(round(time.time() * 1000))
  imgValMean = cv2.mean(img)
  print("modified picture means: ")
  print(imgValMean)

  #cv2.imwrite( "./modImg/img1.jpg", img );
  return img

def getWhiteThings(baseImg): 
  ######## táblák és fejek
  baseImg_hsv = cv2.cvtColor(baseImg, cv2.COLOR_BGR2HSV)
  kernel = np.ones((5,5),np.uint8)
  hsv_color1 = np.asarray([0, 0, 101])   
  hsv_color2 = np.asarray([70, 137, 255])   

  #hsv_color3 = 

  white_things = cv2.inRange(baseImg_hsv, hsv_color1, hsv_color2)


  
  w_t_opened = cv2.morphologyEx(white_things, cv2.MORPH_OPEN, kernel, iterations=1)
  w_t_closed = cv2.morphologyEx(w_t_opened, cv2.MORPH_CLOSE, kernel, iterations=3)

  return white_things

def getWhiteThings2(baseImg): 
  ######## táblák és fejek
  baseImg_hsv = cv2.cvtColor(baseImg, cv2.COLOR_BGR2HSV)
  kernel = np.ones((5,5),np.uint8)
  hsv_color1 = np.asarray([0, 0, 101])   
  hsv_color2 = np.asarray([255, 137, 255])   

  #hsv_color3 = 

  white_things = cv2.inRange(baseImg_hsv, hsv_color1, hsv_color2)

  w_t_opened = cv2.morphologyEx(white_things, cv2.MORPH_OPEN, kernel, iterations=1)
  w_t_closed = cv2.morphologyEx(w_t_opened, cv2.MORPH_CLOSE, kernel, iterations=3)

  return white_things
  


def analyzeContours(contoursAndRectangles, whiteThings, baseImg):
  things = np.zeros((480,640,1), np.uint8)
  infos = np.zeros((480,640,1), np.uint8)
  cactiRectangles = []
  trafficSignRectangles = []
  for i in contoursAndRectangles:
    if(i[0] != [0, 0, 0, 0]):
      contour = i[1]
      thisThing = np.zeros((480,640,1), np.uint8)

      cv2.drawContours(things, contour, 0, (255,255,255), -1)

      cv2.drawContours(thisThing, contour, 0, (255,255,255), -1)
      selectedThing = cv2.bitwise_and(whiteThings, whiteThings, mask = thisThing)
      avg = cv2.mean(selectedThing, mask=thisThing)
      if(avg[0] > 197):
        #szóval ez egy kaktuszfej
        x1, y1, x2, y2 = calculateCactiWrapper(i[0])
        cactiRectangles.append({
          "cactiHead": i[0],
          "cactiHeadContour": contour,
          "cactiWrapper": [x1,y1,x2,y2]
        })
        cv2.rectangle(baseImg, (x1, y1), (x2, y2), (0,0,255),1)
      elif(avg[0] > 20):
        #szóval ez egy tábla
        x1 = i[0][0]
        y1 = i[0][1]
        x2 = i[0][0]+i[0][2]
        y2 = i[0][1]+i[0][3]
        trafficSignRectangles.append([x1, y1, x2, y2])
        cv2.rectangle(baseImg, (x1, y1), (x2, y2), (255,0,0),1)
      cv2.putText(infos, str(avg), (i[0][0], i[0][1]), font, 1, (255))    #i[0][0] és i[0][1] x és y

  res = cv2.bitwise_and(whiteThings, whiteThings, mask = things)
  return res, infos, cactiRectangles, trafficSignRectangles



def calculateDistance(x1, y1, x2, y2):
  return math.sqrt((x1-x2)**2+(y1-y2)**2)


#a feladata, hogy a detektált objektumokat tovább szűrje, hogy a hibákat eltávolítsa
def postProcessObjects(cactis, trafficSigns, vehicles):
  print("asd")

  ###### kaktuszok átnézése, a kicsi fejek, és a hátoldali fej detektálás kiszűrése
  smallHeadsRemoved = []

  for i in cactis:
    if(i["cactiHead"][3]>20):
      smallHeadsRemoved.append(i)

  headsOnBackRemoved = []
  if(len(smallHeadsRemoved) == 1):
    headsOnBackRemoved.append(i)
  else:
    for i in smallHeadsRemoved:
      isItAWrongHead = False
      for j in smallHeadsRemoved:
        distance = calculateDistance(i["cactiHead"][0], i["cactiHead"][1], j["cactiHead"][0], j["cactiHead"][1])
        if(distance < 150):
          area_i = i["cactiHead"][2] * i["cactiHead"][3]
          area_j = j["cactiHead"][2] * j["cactiHead"][3]
          if(area_i < area_j):  #ha ez feltételezhetően nem egy jó fej
            isItAWrongHead = True
      if(not isItAWrongHead):
        headsOnBackRemoved.append(i)


  ##összes suv, és összes plane kiválogatása
  SUVs = []
  planes = []
  trucks = []

  for i in vehicles:
    if(i["type"] == "SUV" or i["type"] == "plane"): #meg kell nézni, hogy milyen közel van a kaktuszhoz, mivel lehet, hogy cserép
      print("DISTANCE TEST:")
      isVehicle = True
      for j in headsOnBackRemoved:
        cactiHeadMid_x = j["cactiHead"][0] + j["cactiHead"][2]/2
        cactiHeadMid_y = j["cactiHead"][1] + j["cactiHead"][3]/2
        SUVMid_x = (i["rect"][0] + i["rect"][2]) / 2
        SUVMid_y = (i["rect"][1] + i["rect"][3]) / 2
        distance = calculateDistance(cactiHeadMid_x, cactiHeadMid_y, SUVMid_x, SUVMid_y)
        print(distance)
        if distance < 100:
          isVehicle = False
      if isVehicle:
        if(i["type"]== "plane"):
          planes.append(i)
        elif(i["type"]== "SUV"):
          SUVs.append(i)
    elif(i["type"] == "truck"):
      trucks.append(i)
    
  if(len(SUVs)>1):
    realSUV = {}
    maxContourLength = 0
    for i in SUVs:
      if i["contourLength"] > maxContourLength:
        realSUV = i
        maxContourLength = i["contourLength"]
    SUVs = []
    SUVs.append(realSUV)
  
  if(len(trucks) > 1):
    realTruck = {}
    maxContourLength = 0
    for i in trucks:
      if i["contourLength"] > maxContourLength:
        realTruck = i
        maxContourLength = i["contourLength"]
    trucks = []
    trucks.append(realTruck)

  #a darut ne érzékelje plane-nek
  if(len(planes)>0):
    newPlanes = []
    for i in planes: 
      if(len(trucks) >0):
        truckMid_x = (trucks[0]["rect"][0] + trucks[0]["rect"][2]) / 2
        truckMid_y = (trucks[0]["rect"][1] + trucks[0]["rect"][3]) / 2
        planeMid_x = (i["rect"][0] +i["rect"][2]) / 2
        planeMid_y = (i["rect"][1] + i["rect"][3]) / 2
        distance = calculateDistance(truckMid_x, truckMid_y, planeMid_x, planeMid_y)
        if(distance<50):  #ha túl közel van, a truck-hoz, akkor ez egy daru, continue
          continue
      newPlanes.append(i)
    planes = newPlanes
  
  if(len(planes) > 1):
    realPlane = {}
    maxContourLength = 0
    for i in planes:
      if i["contourLength"] > maxContourLength:
        realPlane = i
        maxContourLength = i["contourLength"]
    planes = []
    planes.append(realPlane)

#################################### csak a wrapper visszaadása
  newCactis = []
  newVehicles = [] #egylőre teszt, itt majd benne lesz a típus is
  for i in headsOnBackRemoved:
    #newCactis.append(i["cactiWrapper"])
    newCactis.append(i)
  for i in planes:
    newVehicles.append({
      "type": 2,
      "rect":i["rect"]
    })
  for i in SUVs:
    newVehicles.append({
      "type": 0,
      "rect":i["rect"]
    })
  for i in trucks:
    newVehicles.append({
      "type": 1,
      "rect":i["rect"]
    })
  return newCactis, trafficSigns, newVehicles


def calculateCactiWrapper(cactiHeadWrapper):
  x1 = max(cactiHeadWrapper[0] - 20, 0)
  y1 = max(cactiHeadWrapper[1] - 20, 0)
  x2 = min(cactiHeadWrapper[0]+cactiHeadWrapper[2] + 20, 640)
  y2 = min(cactiHeadWrapper[1]+cactiHeadWrapper[3] + 100, 480)
  return [x1, y1, x2, y2]

def detectVehicles(img):
  vehicleList = []
  img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  cv2.rectangle(img_hsv,(0,0),(640,160),(0,0,0),-1) #a felső 1/3 fekete
  hsv_color1 = np.asarray([0, 155, 59])   
  hsv_color2= np.asarray([10, 255, 255])   
  plane = cv2.inRange(img_hsv, hsv_color1, hsv_color2)

  _, th = cv2.threshold(plane, thresh=10, maxval=255, type=cv2.THRESH_BINARY)
  kernel = np.ones((5,5),np.uint8)
  th_opened = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=2)

  contours, hierarchy = cv2.findContours(th_opened, 1, 2)
  for cnt in contours:
    contourLength = cv2.arcLength(cnt, True)
    if contourLength>200:
      contourArea = cv2.contourArea(cnt, False)
      print(">>> Vehicle contour test:")
      print(contourLength)
      print(contourArea)
      
      areaPerLength = contourArea / contourLength
      print("contour area per length "+str(areaPerLength))
      x,y,w,h = cv2.boundingRect(cnt)
      boundingRectArea = w*h
      areaPerBoundingRectArea = contourArea / boundingRectArea
      print("areaPerBoundingRectArea"+str(areaPerBoundingRectArea))
      if(areaPerLength >7.5 and areaPerBoundingRectArea > 0.48 and contourArea>2500):  #! ez egy SUV
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        vehicleList.append({
          "contourLength" : contourLength,
          "type": "SUV",
          "rect":[x,y,x+w,y+h]
        })
        text = "SUV" + str(areaPerLength)
        cv2.putText(img, text, (x, y), font, 1, (0,0,255))
      elif (areaPerLength>6 and areaPerBoundingRectArea < 0.48): #!ez egy plane
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        vehicleList.append({
          "contourLength" : contourLength,
          "type": "plane",
          "rect":[x,y,x+w,y+h]
        })
        text = "Plane" + str(areaPerLength)
        cv2.putText(img, text, (x, y), font, 1, (0,0,255))
      else:
        text = str(areaPerLength) 
        cv2.putText(img, text, (x, y), font, 1, (0,0,255))
      cv2.drawContours(img, [cnt], 0, (0,255,0), 2)

  
  hsv_color1 = np.asarray([13, 183, 134])   
  hsv_color2= np.asarray([29, 255, 255])   
  truck = cv2.inRange(img_hsv, hsv_color1, hsv_color2)
  
  _, th = cv2.threshold(truck, thresh=10, maxval=255, type=cv2.THRESH_BINARY)
  kernel = np.ones((5,5),np.uint8)
  th_opened = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=2)
  contours, hierarchy = cv2.findContours(th_opened, 1, 2) 

  for cnt in contours:
    contourLength = cv2.arcLength(cnt, True) 
    contourArea = cv2.contourArea(cnt, False) 
    if contourLength>200 and contourArea>1500 and contourArea<6000: #legyen kisebb mint 6k?
      print("truck contour area:")
      print(contourArea)
      x,y,w,h = cv2.boundingRect(cnt)
      cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
      vehicleList.append({
        "contourLength" : contourLength,
        "type": "truck",
        "rect":[x,y,x+w,y+h]
      })
      cv2.putText(img, "Truck", (x, y), font, 1, (0,0,255))
  return th_opened, vehicleList



def getClosedArcLength(curve): 
  return cv2.arcLength(curve, True)

def isContainsPoint(point_x, point_y, rect):
  if (point_x >= rect[0] and point_x <= (rect[0]+rect[2])) and (point_y >= rect[1] and point_y <=rect[1]+rect[3]):
    print("_________found a collision:")
    print("this contains: ")
    print(rect)
    print("this point:")
    print([point_x , point_y])
    return True
  else:
    return False

def analyzeFaces(cactis, faceImg):
  allFaces = np.zeros((480,640,1), np.uint8)
  grayImg = cv2.cvtColor(faceImg, cv2.COLOR_BGR2GRAY)
  newCactis = []    #ebben benne lesz a típus is
  for cacti in cactis:
    faceMask = np.zeros((480,640,1), np.uint8)
    cv2.drawContours(faceMask, cacti["cactiHeadContour"], 0, (255,255,255), -1)
    #esetleg hist eq
    #itt még a feldolgozást ki kell találni
    avg = cv2.mean(grayImg, mask=faceMask)
    thresholdVal = avg[0]*0.9 #ki kell találni, hogy mi legyen
    selectedFace = cv2.bitwise_and(grayImg, grayImg, mask = faceMask)
    #cv2.equalizeHist(img)
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])    #élesítés
    sharpenedFace = cv2.filter2D(selectedFace, -1, kernel)

    _, th = cv2.threshold(sharpenedFace, thresh=thresholdVal, maxval=255, type=cv2.THRESH_BINARY)

    th_inv = 255 - th
    #kernel = np.ones((3,3),np.uint8)
    #th_closed = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=1)
    #inv_th = 255 - th
    #blur = cv2.medianBlur(inv_th, 3)
    #th = 255-blur

    cnt, hierarchy = cv2.findContours(th_inv, 1 ,2)
    sortedContours = sorted(cnt, key = lambda x:cv2.arcLength(x, True))
    contourToShow = sortedContours[-1]
    #cv2.drawContours(faceImg, [contourToShow], 0, (255, 0, 0), 1)
    maskedMouthTest = np.zeros((480,640,1), np.uint8)
    isSmile = False
    for i in reversed(sortedContours):  #cél, hogy megtalálja a szájat
      contourLength = cv2.arcLength(i, True)
      contourArea = cv2.contourArea(i, False)
      if(contourLength > 40 and contourArea<150 and contourLength<150):   #a száj megkeresése, ez többnyire jó
        cv2.drawContours(faceImg, [i], 0, (255, 0, 0), 1)
        print("contour length:")
        print(contourLength)
        print("contour area:")
        print(contourArea)
        x, y, w, h = cv2.boundingRect(i)
        ##! tesztelni kell, hogy mit tekint mouth-nak
        test_mask = np.zeros((480,640,1), np.uint8)
        cv2.rectangle(test_mask, (x,y), (x+w, y+h), (255, 255, 255), -1)
        maskedMouthTest = cv2.bitwise_and(th, th, mask=test_mask)

        mouth = th[y:y+h, x:x+w]
        if(w>15 and h>=5):
          subArray1 = mouth[0:5, 0:5]
          subArray2 = mouth[0:5, w-5:w]
          unique, counts = np.unique(subArray1, return_counts=True)
          for index, val in enumerate(unique):
            if(val == 0):
              if(counts[index] > 5):
                print("count index test:")
                print(counts[index])
                isSmile = True
          unique, counts = np.unique(subArray2, return_counts=True)
          for index, val in enumerate(unique):
            if(val == 0):
              if(counts[index] > 5):
                print("count index test:")
                print(counts[index])
                isSmile = True
        break
      else:   #ha nem talált szájat (ilyenkor van, hogy beleér az aljába, tehát sad)
        isSmile = False
    
    print("is smile?")
    print(isSmile)
    allFaces = cv2.bitwise_or(allFaces, th)

    #TODO szemöldök detektálás
    faceFeatureWrappers = []
    isIntersectDetected = False
    for i in sortedContours:
      if(cv2.arcLength(i, True)>6 and cv2.arcLength(i, True)<40): 
        x, y, w, h = cv2.boundingRect(i)
        faceFeatureWrappers.append([x,y,w,h])
    for i in range(len(faceFeatureWrappers)):     #megnézi, hogy az i négyzetben benne van-e a j négyzet valamelyik sarka
      if isIntersectDetected: break
      for j in range (len(faceFeatureWrappers)):
        if(i == j): continue
        if isIntersectDetected: break 
        p = faceFeatureWrappers[j]
        points = [[p[0], p[1]], [p[0]+p[2], p[1]], [p[0]+p[2], p[1]+p[3]], [p[0], p[1]+p[3]]]
        for k in points:
          if isContainsPoint(k[0], k[1], faceFeatureWrappers[i]):
            isIntersectDetected = True
            break
        #if(isContainsPoint)
        #print(i & j)
    print("has eyebrows? : ")
    print(isIntersectDetected)

    ##################################### kiértékelés
    cactiType = 0
    if(isSmile and isIntersectDetected):
      print("It is evil")
      cactiType = 3
    elif(isSmile and not isIntersectDetected):
      print("It is happy :)")
      cactiType = 0
    elif(not isSmile and not isIntersectDetected):
      print("It is sad :(")
      cactiType = 1
    elif(not isSmile and isIntersectDetected):
      print("It is angry")
      cactiType = 2

    newCacti = cacti
    cacti["type"] = cactiType
    newCactis.append(cacti)

  #kaktusz for ciklus vége
  #faceImg = cv2.bitwise_and(faceImg, faceImg, mask = faceMask)
  return newCactis, allFaces




def createMyAnswers():
  myAnswers = {}
  for subdir, dirs,  files in os.walk("./HW"):
    for file in files:
      filepath = subdir + os.sep + file
      if(subdir[-3:] == "rgb"):
        print("dir test:")
        print(subdir[-6:-4])
        baseImg = cv2.imread(filepath)
        baseImgCpy = baseImg.copy()
        baseImgCpy2 = baseImg.copy()
        cactiFaceImg = baseImg.copy()
        objectContours, contoursAndRectangles = getContours(baseImg)
        #modColor = colorTransform(baseImgCpy)
        whiteThings = getWhiteThings2(baseImgCpy)
        res, infos, cactis, trafficSigns = analyzeContours(contoursAndRectangles, whiteThings, baseImgCpy)
        test, vehicles = detectVehicles(baseImgCpy2)


        cactis, trafficSigns, vehicles = postProcessObjects(cactis, trafficSigns, vehicles)
        cactis, _ = analyzeFaces(cactis, cactiFaceImg)
        
        myObjects = []
        for i in cactis:
          u = (i["cactiWrapper"][0]+i["cactiWrapper"][2])//2
          v = (i["cactiWrapper"][1]+i["cactiWrapper"][3])//2
          w = i["cactiWrapper"][2] - i["cactiWrapper"][0]
          h = i["cactiWrapper"][3] - i["cactiWrapper"][1] 
          myObject = [u, v, w, h, 2, i["type"], 0, 0, 0]
          myObjects.append(myObject)

        for i in trafficSigns:
          u = (i[0]+i[2])//2
          v = (i[1]+i[3])//2
          w = i[2] - i[0]
          h = i[3] - i[1] 
          myObject = [u, v, w, h, 0, 0, 0, 0, 0]
          myObjects.append(myObject)

        for i in vehicles:
          u = (i["rect"][0]+i["rect"][2])//2
          v = (i["rect"][1]+i["rect"][3])//2
          w = i["rect"][2] - i["rect"][0]
          h = i["rect"][3] - i["rect"][1] 
          myObject = [u, v, w, h, 1, i["type"], 0, 0, 0]
          myObjects.append(myObject)

        pose = [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]  
        myPred = {
           'poses' :[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
           'objects' : myObjects
        }
        myAnswers['HW/'+subdir[-6:-4]+'/rgb/'+file]  = myPred
  return myAnswers



#################################################### MAIN ####################################################
#testOnSingleImg()

###### Answer and evaluation ######
#myAnswers = createMyAnswers()
#print("task 1 finished")
#rint(myAnswers)
#evaluate(myAnswers)
#pickle.dump(myAnswers, open('HW/ourpredictions.pickle', 'wb'))   

#filehandler = open(b"my_answer.pickle","wb")
#pickle.dump(myAnswers,filehandler)

# Task 2: (deep learning)
# BASIC: Use a deep learning algorithm to classify traffic signs. The package provided includes a training and validation database of 32x32 RGB images.
# OUTPUT: traffic signs / pictures (without 3 missing)

# HARDCORE: Of the 55 possible traffic signs, 3 are missing from the training and test datasets. ('X - Priority', 'X - Turn left', 'X - Turn right') 
# As a result, the neural net trained in task 2 will not be able to classify them properly. Extend your neural network to classify these as well.
# OUTPUT: traffic signs / pictures (all)

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch import optim

# Convolutional module (Conv+ReLU+BatchNorm)
class Conv(nn.Module):
  def __init__(self, in_channels, channels,kernel_size=3, stride=1):   #kernel_size=3
     super(Conv, self).__init__()
     self.conv = nn.Conv2d(in_channels, channels, kernel_size, stride=stride, padding=kernel_size//2, bias=False)   #padding=1
     self.bn = nn.BatchNorm2d(channels)
  def forward(self,x):
     return self.bn(torch.relu(self.conv(x)))
  

class ConvNet(nn.Module):
  def __init__(self, base_channels=16, in_channels=3, num_classes=55):  #in_channels=4
      super(ConvNet, self).__init__()   
      
      self.c11 = Conv(in_channels, base_channels)
      self.c12 = Conv(base_channels, base_channels)
      self.d1 = Conv(base_channels, base_channels*2, stride=2)

      self.c21 = Conv(base_channels*2, base_channels*2)
      self.c22 = Conv(base_channels*2, base_channels*2)
      self.d2 = Conv(base_channels*2, base_channels*4, stride=2)
        
      self.c31 = Conv(base_channels*4, base_channels*4)
      self.c32 = Conv(base_channels*4, base_channels*4)
      self.d3 = Conv(base_channels*4, base_channels*8, stride=2)
        
      self.c41 = Conv(base_channels*8, base_channels*8)
      self.c42 = Conv(base_channels*8, base_channels*8)
      self.d4 = Conv(base_channels*8, base_channels*16, stride=2)
        
      self.c51 = Conv(base_channels*16, base_channels*16)
      self.c52 = Conv(base_channels*16, base_channels*16)
      self.d5 = Conv(base_channels*16, base_channels*32, stride=2)
      # Input image is 32x32 -> after 5 downscaling the activation map is 1x1
      
      # Classifier is a convolution that produces num_classes class scores
      self.classifier = nn.Conv2d(base_channels*32,num_classes, 1) #kernel_size=1

  def forward(self,x):
        # Class all the layers
        x = self.d1(self.c12(self.c11(x)))
        x = self.d2(self.c22(self.c21(x)))
        x = self.d3(self.c32(self.c31(x)))
        x = self.d4(self.c42(self.c41(x)))
        x = self.d5(self.c52(self.c51(x)))
        return torch.squeeze(self.classifier(x))


haveCuda = torch.cuda.is_available()

#DATA AUGMENTATION
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49139968, 0.48215827, 0.44653124),
                         (0.24703233, 0.24348505, 0.26158768))
     #transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)) # mean = 0 and standard-deviation = 1
])

transform = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    # Random perturbance of brightness, contrast and color
    transforms.ColorJitter(brightness=0.3,contrast=0.3,saturation=0.3,hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.49139968, 0.48215827, 0.44653124),
                         (0.24703233, 0.24348505, 0.26158768))
    #transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629)) # mean = 0 and standard-deviation = 1
])

#DATASETS
#trainSet = torchvision.datasets.ImageFolder(root="/content/trafficSignsHW/trainFULL", transform=transform)
#testSet = torchvision.datasets.ImageFolder(root="/content/trafficSignsHW/testFULL", transform=transform_val) #is_valid_file=None

#trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=32, shuffle=True) #sampler overfitting kevés adatra bs=128
#testLoader = torch.utils.data.DataLoader(testSet, batch_size=32, shuffle=False)


def createNet():
    net = ConvNet()
    if haveCuda:
        net = net.cuda()
    return net

def createLoss():
    return nn.CrossEntropyLoss()

# create optimizer
def createOptimizer():
    return optim.SGD(net.parameters(), lr=1e-2, momentum=0.5)  # lr=1e-1, momentum=0.9,nesterov=True, weight_decay=1e-4

numEpoch =20

def createScheduler():
    return optim.lr_scheduler.CosineAnnealingLR(optimizer,numEpoch,eta_min=1e-2)


#Progress bar
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))



def train(epoch):

    # variables for loss
    running_loss = 0.0
    correct = 0.0
    total = 0

    # set the network to train (for batchnorm and dropout)
    net.train()

    # Create progress bar
    bar = display(progress(0, len(trainLoader)), display_id=True)

    for i, data in enumerate(trainLoader, 0):
        
        inputs, labels = data
        if haveCuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        # Forward
        outputs = net(inputs)
        # Loss
        loss = criterion(outputs, labels)  #loss = torch.nn.functional.nll_loss(outputs, labels)
        # Backpropagation
        loss.backward()
        # Gradient method
        optimizer.step()

        # Do not include these steps in the computational graph
        with torch.no_grad():
            # Accumulate loss
            running_loss += loss.item()
            # Get indices of the largest goodness values
            _, predicted = torch.max(outputs, 1)
            # Count how many of the predictions equal the labels
            correct += predicted.eq(labels).sum().item()
            # Accumulate number of total images seen
            total += labels.shape[0]   ####

        # Progress bar
        bar.update(progress(i+1, len(trainLoader)))

    # return loss and accuracy
    tr_loss = running_loss / i
    tr_corr = correct / total * 100
    print("Train epoch %d loss: %.3f correct: %.2f" % (epoch + 1, running_loss / i, tr_corr))
    return tr_loss,tr_corr

# Function for validating a single epch
def val(epoch):

    # variables for loss
    running_loss = 0.0
    correct = 0.0
    total = 0

    # set the network to eval  (for batchnorm and dropout)
    net.eval()

    # Create progress bar
    bar = display(progress(0, len(testLoader)), display_id=True)

    for i, data in enumerate(testLoader, 0):
        # get the inputs
        inputs, labels = data
        if haveCuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        # Do not include these steps in the computational graph
        with torch.no_grad():
            # Forward
            outputs = net(inputs)
            # Compute loss
            loss = criterion(outputs, labels)

            # Compute statistics, just like before
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += predicted.eq(labels).sum().item()
            total += labels.shape[0]

        bar.update(progress(i+1, len(testLoader)))

    # return loss and accuracy
    val_loss = running_loss / i
    val_corr = correct / total * 100
    print("Test epoch %d loss: %.3f correct: %.2f" % (epoch + 1, running_loss / i, val_corr))
    return val_loss,val_corr

transform = transforms.Compose([transforms.ToTensor()]) 

#Read model
model = torch.load("./model.pth", map_location=torch.device('cpu'))
model.eval()

def prepend(list, str): 
    list = [str + i for i in list] 
    return(list) 

def sorted_nicely( l ):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

def hf_DL_task(path_of_source_pickle,path_of_destination_pickle):
  # Define used arrays and variables
  namesRGB = []

  # Get folders
  myFolderList = sorted([f.path for f in os.scandir("/content/HW") if f.is_dir()])

  # Put all RGB images to an array from "gx" folders
  for element in myFolderList:
    
    # Get and sort list of objects
    actualRGB = (sorted_nicely(glob.glob1(element + "/rgb", "*.jpg")))
    
    # Add path to all images
    path = element.replace('/content/', '')
    actualRGB = prepend(actualRGB, path + '/rgb/')
    
    
    # Put names to the big global array
    namesRGB = namesRGB + actualRGB
  
  # Open source pickle what contains data from previous tasks
  file = open(path_of_source_pickle,'rb')
  ourPrediction = pickle.load(file)
  
  # Loop throught all images
  for photoIndex in range(len(namesRGB)):
    # Get the name of the actual RGB picture
    actualPic = namesRGB[photoIndex]
    rgbPicture = cv2.imread(actualPic, 0)

    for actObject in ourPrediction[actualPic]["objects"]:
      # Get center coordinates of the object's bounding box
      u = actObject[0]
      v = actObject[1]
      w = actObject[2]
      h = actObject[3]
      c = actObject[4]
      
      #if Class  name== 'traffic sign'
      if c==0:
        a = int(v-w//2)
        b = int(v+w//2)
        c = int(u-h//2)
        d = int(u+h//2)
        crop_image = rgbPicture[a:b, c:d]
        rs_img=cv2.resize(crop_image, (32,32), interpolation = cv2.INTER_AREA)
        rs_img_rgb= cv2.cvtColor(rs_img,cv2.COLOR_BGR2RGB)
        dl_img = transform(rs_img_rgb)  # Preprocess image
        dl_img = dl_img.unsqueeze(0)  # Add batch dimension
        if haveCuda:
          dl_img=dl_img.cuda()
        output = model(dl_img)  # Forward pass
        _, pred = torch.max(output, 0)  # Get predicted class
        
        #Put subclassName to our prediction
        actObject[5] = pred.item()
  
  # Save results to our predictions
  pickle.dump(ourPrediction, open(path_of_destination_pickle, 'wb'))


#5 Task 3: (3d vision)
# BASIC: 3D Vision: Determine the 3D positions of the object of interest relative to the camera. Use the center of an object's bounding box to determine the position on the image.
# OUTPUT: x,y and z coordinates (m) of objects / pictures
import os
import glob
import re
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

def prepend(list, str): 
    list = [str + i for i in list] 
    return(list) 

def sorted_nicely( l ):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

def calculate_distance_2d(x1, y1, x2, y2):
    return np.sqrt(np.square(x2-x1)+np.square(y2-y1))

#TEST ARGUMENTS
# Source: 'HW/annotations.pickle'
# Destionation: "HW/ourpredictions.pickle"
def hf_3d_task_1(path_of_source_pickle, path_of_destination_pickle):
  # Define used arrays and variables
  namesRGB = []
  namesDEPTH = []
  
  # Set up random seed for BLUR
  cv2.setRNGSeed(112)

  # Get folders
  myFolderList = sorted([f.path for f in os.scandir("/content/HW") if f.is_dir()])

  # Put all RGB and DEPTH images to an array from "gx" folders
  for element in myFolderList:
    
    # Get and sort list of objects
    actualRGB = (sorted_nicely(glob.glob1(element + "/rgb", "*.jpg")))
    actualDEPTH = (sorted_nicely(glob.glob1(element + "/depth", "*.png")))
    
    # Add path to all images
    path = element.replace('/content/', '')
    actualRGB = prepend(actualRGB, path + '/rgb/')
    actualDEPTH = prepend(actualDEPTH, path + '/depth/')
    
    # Put names to the big global array
    namesRGB = namesRGB + actualRGB
    namesDEPTH = namesDEPTH + actualDEPTH

  # DIRTY MODE ON
  # Fix yaml... :'( -> except the !dtEmptpy function throw error...
  with open('/content/HW/calibration.yaml', 'r') as in_file:
      buf = in_file.readlines()
  with open('/content/HW/calibration_fixed.yaml', 'w') as out_file:
      for line in buf:
          if line == "   cols: 3\n":
              line = line + "   dt: d\n"
          if line == "   cols: 5\n":
              line = line + "   dt: d\n"
          out_file.write(line)
  #  DIRTY MODE OFF

  # Read calibration data
  fs = cv2.FileStorage('/content/HW/calibration_fixed.yaml', cv2.FILE_STORAGE_READ)

  # Process important nodes from calibration data
  cameraMatrix = fs.getNode("camera_matrix").mat()
  distorsionCoefs = fs.getNode("distortion_coefficients").mat()
  distorsionModel = fs.getNode("distortion_model").string()
  picWidth = int(fs.getNode("image_width").real())
  picHeight = int(fs.getNode("image_height").real())

  # Open source pickle what contains data from previous tasks
  file = open(path_of_source_pickle,'rb')
  ourPrediction = pickle.load(file)

  # Decompose camera matrix to get focal lengths expressed in pixel units and principal point
  fx = cameraMatrix[0,0]
  fy = cameraMatrix[1,1]
  cx = cameraMatrix[0,2]
  cy = cameraMatrix[1,2]

  # BAISC TASK
  # Loop throught all images
  for photoIndex in range(len(namesRGB)):
    # Read DEPTH map of the actual image
    depthRaw = cv2.imread(namesDEPTH[photoIndex], -1)

    # Add some BLUR
    depth = cv2.medianBlur(depthRaw, 3)

    # Convert depth map to np.float32
    depth = np.float32(depth)

    # Get the name of the actual RGB picture
    actualPic = namesRGB[photoIndex]

    # Calculate world coordinates for all objects
    for actObject in ourPrediction[actualPic]["objects"]:
      # Get center coordinates of the object's bounding box
      u = actObject[0]
      v = actObject[1]

      # Convert 2D points to 3D space (and get values in METERs)
      ourX = ((depth[v, u] / fx) * (u - cx)) / 1000
      ourY = ((-1)*(depth[v, u] / fy) * (v - cy))/1000
      ourZ = depth[v, u]/1000

      # Put 3D coordinates to our prediction (fill space in list or append, if space not exists)
      if len(actObject) < 7:
        actObject.append(ourX)
      else:
        actObject[6] = ourX
      if len(actObject) < 8:
        actObject.append(ourY)
      else:
        actObject[7] = ourY
      if len(actObject) < 9:
        actObject.append(ourZ)
      else:
        actObject[8] = ourZ

  # Save results to our predictions
  pickle.dump(ourPrediction, open(path_of_destination_pickle, 'wb'))
  
  # Load results to another isntance
  file = open(path_of_destination_pickle,'rb')
  ourHCPredictions = pickle.load(file)
  
  # HARDCORE TASK
  
  # Define the scannable part (half)size and define mats
  rangeOfScan = 33
  # 3D points of the actual picture
  obj = np.arange(rangeOfScan*rangeOfScan*3, dtype='float32').reshape(rangeOfScan*rangeOfScan,3,1)
  # 2D points of the previous picture
  img = np.arange(rangeOfScan*rangeOfScan*2, dtype='float32').reshape(rangeOfScan*rangeOfScan,2,1)
  
  # Copy ourHCPrediction (to keep it clean)
  copyOfPred = ourHCPredictions

  # Loop throught all images
  for actPic in namesRGB:
    # Initialize INVALID minimum parameters to the minimum search
    minimumVal = 10000
    minimumIdx = 0

    # Calculate all element's distance from center and choose the nearest
    for actObject in copyOfPred[actPic]["objects"]:
      # Get center coordinatees
      actU = actObject[0]
      actV = actObject[1]

      # Measure distance from center
      distanceFromCenter = calculate_distance_2d(actU, actV, 320, 240)

      # Minimum search
      if (minimumVal > distanceFromCenter):
        minimumVal = distanceFromCenter
        minimumIdx = copyOfPred[actPic]["objects"].index(actObject)
      
      # Save minimum index to the dictionary
      copyOfPred[actPic]["minidx"] = minimumIdx

  # Loop throught all images
  for photoIndex in range(len(namesRGB)):
    # Get ACTUAL picture name
    actualPic = namesRGB[photoIndex]
    
    # Run if we have pictures from the same video
    if ("/1.jpg" not in actualPic):
      # Get PREVIOUS picture name and ACTUAL depth map
      prevPic = namesRGB[photoIndex - 1]
      prevPicDepth = namesDEPTH[photoIndex - 1]
      
      # Get the index of the nearest object from the center
      minIdxAct = copyOfPred[actualPic]["minidx"]
      minIdxPrev = copyOfPred[prevPic]["minidx"]
      
      # Measure range of scan from center of the ACTUAL picture and add the left bottom corner point
      limitLeft = copyOfPred[actualPic]["objects"][minIdxAct][1] - int((rangeOfScan-1)/2) 
      limitBottom = copyOfPred[actualPic]["objects"][minIdxAct][0] - int((rangeOfScan-1)/2) 

      # Load depth map and do same conditioning as before 
      depthRaw = cv2.imread(prevPicDepth, -1)
      depth = cv2.medianBlur(depthRaw, 3)
      depth = np.float32(depth)
      
      # Convert 2D coordinates to 3D space in the scanned RANGE and save in ACTUAL PICTURE
      indexOfScannedRange = 0
      for u in range(limitLeft, limitLeft + rangeOfScan):
        for v in range(limitBottom, limitBottom + rangeOfScan):
          obj[indexOfScannedRange][0] = ((depth[v, u] / fx) * (u - cx)) / 1000
          obj[indexOfScannedRange][1] = ((-1)*(depth[v, u] / fy) * (v - cy))/1000
          obj[indexOfScannedRange][2] = depth[v, u]/1000
          indexOfScannedRange += 1
      
      # Try to find a pair to this object or use the nearest from the center in PREVIUOS picture
      counterObj = 0
      for prevObj in copyOfPred[prevPic]["objects"]:
        # Get comparable types of the objects
        actType = copyOfPred[actualPic]["objects"][minIdxAct][4]
        actSubType = copyOfPred[actualPic]["objects"][minIdxAct][5]
        prevType = prevObj[4]
        prevSubType = prevObj[5]

        # Change nearest object to the similar if possible
        if actType == prevType and actSubType == prevSubType:
          minIdxPrev = counterObj
        counterObj += 1

      # Measure range of scan from center of the PREVIOUS picture and add the left bottom corner point
      limitLeft = copyOfPred[prevPic]["objects"][minIdxPrev][1] - int((rangeOfScan-1)/2)
      limitBottom = copyOfPred[prevPic]["objects"][minIdxPrev][0] - int((rangeOfScan-1)/2)

      # Save 2D coordinates in ACTUAL PICTURE
      indexOfScannedRange = 0
      for u in range(limitLeft, limitLeft + rangeOfScan):
        for v in range(limitBottom, limitBottom + rangeOfScan):
          img[indexOfScannedRange][0] = u
          img[indexOfScannedRange][1] = v
          indexOfScannedRange += 1
      
      # Put 3D and 2D pairs to the PnP problem solver ang get rotation vector and translation mat
      returnVal, rotVector, translationMat = cv2.solvePnP(obj, img, cameraMatrix, distorsionCoefs, flags=cv2.SOLVEPNP_ITERATIVE) 

      # PNP solved
      if returnVal == True:
        # Calculate rotation with Rodrigues formula
        rotMat = cv2.Rodrigues(rotVector)[0]
        
        # Assemble pose mat
        poseMat = np.hstack((-rotMat.T,translationMat))

        # Save relative to the previous pose
        ourHCPredictions[actualPic]["poses"] = np.abs(poseMat.reshape(12))*ourHCPredictions[prevPic]["poses"]

      # PNP error -> Put identity mat to the prediction
      else:
        ourHCPredictions[actualPic]["poses"] = np.hstack((np.eye(3),np.zeros(3).reshape(3, 1))).reshape(12)
  
  # Save dictionary
  pickle.dump(ourHCPredictions, open(path_of_destination_pickle, 'wb'))   

# Call Task3 function
# hf_3d_task_1('HW/annotations.pickle', "HW/ourpredictions.pickle")
#hf_3d_task_1('HW/ourpredictions.pickle', "HW/ourpredictions.pickle")


In [54]:
from HW.evaluate import evaluate

# Task 1
#myAnswers = createMyAnswers()
#pickle.dump(myAnswers, open('HW/ourpredictions.pickle', 'wb'))

#Task 2
hf_DL_task('HW/ourpredictions.pickle',"HW/ourpredictions.pickle")

# Task 3
hf_3d_task_1('HW/ourpredictions.pickle', "HW/ourpredictions.pickle")

file = open('HW/ourpredictions.pickle','rb')
pre = pickle.load(file)

evaluate(pre)

Task 1: 0.8220929370784851
Task 1 HC: 0.5140449438202247
Task 2: 0.7182539682539683
Task 2 HC: 1.0
Task 3: 0.9322575719646632
Task 3 HC: 0.6065633637405526
Total:  4.5932127848578945
